In [ ]:
import numpy as np
dataset = np.load("dataset_moves.npy", allow_pickle=True)
results = np.load("dataset_results.npy", allow_pickle=True)

## Quanti match utili

In [2]:
X = 20
count = sum(len(match) > X for match in dataset)
print(f"Nel dataset ci sono {count} match con più di {X} mosse")

Nel dataset ci sono 0 match con più di 20 mosse


## La n-esima mossa più frequente

In [23]:
from collections import Counter

idx = 0         # which move (column)
num_list = 5    # how many results

idx_moves = []
for match in dataset :
    try:
        idx_moves.append(tuple(match[idx]))
    except: continue
    
move_counter = Counter(idx_moves)
most_common_move, most_common_count = move_counter.most_common(1)[0]

most_common_moves = move_counter.most_common(num_list)

# Stampa le N mosse più frequenti
for move, count in most_common_moves:
    print(f"Mossa: ({move[0].item()}, {move[1].item()}), Occorrenze: {count}")

Mossa: (e3, d3), Occorrenze: 167
Mossa: (e3, f3), Occorrenze: 80
Mossa: (d5, d4), Occorrenze: 62
Mossa: (e3, b3), Occorrenze: 51
Mossa: (e4, d4), Occorrenze: 44


## Quante vittorie, pareggi o sconfitte ?

In [24]:
white_wins_supposed = sum(len(match)%2!=0 for match in dataset)
black_wins_supposed = sum(len(match)%2==0 for match in dataset)

white_wins = sum(res=="W" for res in results)   # White wins
black_wins = sum(res=="B" for res in results)   # Black wins
draws =      sum(res=="D" for res in results)   # Draw
other =      sum(res=="I" for res in results)   # Incomplete result

print(f"Total matches: {len(dataset)}")

print(white_wins_supposed,black_wins_supposed)
print(white_wins_supposed + black_wins_supposed)

print(white_wins,black_wins,draws,other)
print(white_wins + black_wins + draws + other)

Total matches: 869
394 475
869
304 376 77 112
869


## Delete incomplete results

In [25]:
incomplete_idxs = results=="I"
dataset_cleaned = np.delete(dataset, incomplete_idxs, axis=0)
results_cleaned = np.delete(results, incomplete_idxs, axis=0)

In [26]:
white_wins_supposed = sum(len(match)%2!=0 for match in dataset_cleaned)
black_wins_supposed = sum(len(match)%2==0 for match in dataset_cleaned)

white_wins = sum(res=="W" for res in results_cleaned)   # White wins
black_wins = sum(res=="B" for res in results_cleaned)   # Black wins
draws =      sum(res=="D" for res in results_cleaned)   # Draw
other =      sum(res=="I" for res in results_cleaned)   # Incomplete result

print(f"Total matches: {len(dataset_cleaned)}")

print(white_wins_supposed,black_wins_supposed)
print(white_wins_supposed + black_wins_supposed)

print(white_wins,black_wins,draws,other)
print(white_wins + black_wins + draws + other)

Total matches: 757
330 427
757
304 376 77 0
757


In [28]:
np.save("dataset_moves_cleaned.npy", dataset_cleaned)
np.save("dataset_results_cleaned.npy", results_cleaned)

## Qual è la n-esima mossa più vincente ?

In [ ]:
from collections import Counter

idx = 0         # which move (column)
num_list = 5    # how many results (None for all)

idx_moves = []
for i, match in enumerate(dataset_cleaned) :
    try:
        res = results_cleaned[i]
        if (idx % 2 == 0 and res=="W") or (idx % 2 != 0 and res=="B") :
            idx_moves.append(tuple(match[idx]))
    except: continue
    
move_counter = Counter(idx_moves)
most_common_move, most_common_count = move_counter.most_common(1)[0]

most_common_moves = move_counter.most_common(num_list)

# Stampa le N mosse più frequenti
# c = 0   # probability check
for move, count in most_common_moves:
    
    if idx % 2 == 0:
        print(f"Mossa: ({move[0].item()}, {move[1].item()}), Win race: {count / white_wins * 100:.4f}%")
        # c += count / white_wins * 100   # probability check
    else:
        print(f"Mossa: ({move[0].item()}, {move[1].item()}), Win race: {count / black_wins * 100:.4f}%")
        # c += count / black_wins * 100   # probability check
# print(c)    # probability check

Mossa: (e3, d3), Win race: 21.3816%
Mossa: (e3, f3), Win race: 9.5395%
Mossa: (e3, b3), Win race: 6.9079%
Mossa: (f5, f4), Win race: 6.9079%
Mossa: (e4, d4), Win race: 6.5789%
